In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE159472"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE159472"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE159472.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE159472.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE159472.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from DLBCL expression study"
!Series_summary	"We used microarray data sets to determine concordance of microarray vs qRT-PCR based LCOO cell-of-origin for Diffuse large B-cell Lymphome subtypes."
!Series_overall_design	"Three independent sets of samples were processed and analyzed for a total of 180 samples. Training sets were developed to build and develop a DLBCL classification model. A subsequent validation data set was used to validate the classification model."
Sample Characteristics Dictionary:
{0: ['dataset group: Training 1', 'dataset group: Training 2', 'dataset group: Validation'], 1: ['tissue: large B-cell lymphoma tissue'], 2: ['subtype (rmsg100): GCB', 'subtype (rmsg100): ABC']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain microarray data for gene expression
# The series title and summary mention "expression data" and "microarray data sets"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for trait, age, and gender

# For trait (DLBCL subtype)
# Row 2 contains "subtype (rmsg100): GCB" and "subtype (rmsg100): ABC", which are DLBCL subtypes
trait_row = 2

# For age - Not available in the sample characteristics dictionary
age_row = None

# For gender - Not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert DLBCL subtype to binary (0 for GCB, 1 for ABC)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'GCB' in value:
        return 0
    elif 'ABC' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous, but not used in this dataset as age is not available"""
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male), but not used in this dataset as gender is not available"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None and 'clinical_data' in locals():
    # Use geo_select_clinical_features to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM4830213': [0.0], 'GSM4830214': [1.0], 'GSM4830215': [1.0], 'GSM4830216': [1.0], 'GSM4830217': [0.0], 'GSM4830218': [0.0], 'GSM4830219': [0.0], 'GSM4830220': [0.0], 'GSM4830221': [1.0], 'GSM4830222': [1.0], 'GSM4830223': [1.0], 'GSM4830224': [0.0], 'GSM4830225': [0.0], 'GSM4830226': [0.0], 'GSM4830227': [1.0], 'GSM4830228': [1.0], 'GSM4830229': [1.0], 'GSM4830230': [0.0], 'GSM4830231': [0.0], 'GSM4830232': [0.0], 'GSM4830233': [0.0], 'GSM4830234': [0.0], 'GSM4830235': [1.0], 'GSM4830236': [0.0], 'GSM4830237': [0.0], 'GSM4830238': [1.0], 'GSM4830239': [0.0], 'GSM4830240': [1.0], 'GSM4830241': [0.0], 'GSM4830242': [1.0], 'GSM4830243': [0.0], 'GSM4830244': [1.0], 'GSM4830245': [1.0], 'GSM4830246': [1.0], 'GSM4830247': [0.0], 'GSM4830248': [0.0], 'GSM4830249': [1.0], 'GSM4830250': [1.0], 'GSM4830251': [1.0], 'GSM4830252': [0.0], 'GSM4830253': [1.0], 'GSM4830254': [0.0], 'GSM4830255': [1.0], 'GSM4830256': [0.0], 'GSM4830257': [1.0], 'GSM4830258': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Expression data from DLBCL expression study"
Line 1: !Series_geo_accession	"GSE159472"
Line 2: !Series_status	"Public on Dec 07 2020"
Line 3: !Series_submission_date	"Oct 13 2020"
Line 4: !Series_last_update_date	"Jan 04 2021"
Line 5: !Series_pubmed_id	"33385586"
Line 6: !Series_summary	"We used microarray data sets to determine concordance of microarray vs qRT-PCR based LCOO cell-of-origin for Diffuse large B-cell Lymphome subtypes."
Line 7: !Series_overall_design	"Three independent sets of samples were processed and analyzed for a total of 180 samples. Training sets were developed to build and develop a DLBCL classification model. A subsequent validation data set was used to validate the classification model."
Line 8: !Series_type	"Expression profiling by array"
Line 9: !Series_contributor	"Rajiv,,Dua"
Found table marker at line 61
First few lines after marker:
"ID_REF"	"GSM4830213"	"GSM4830214"	"GSM4830215"	"GSM4830216"	"GSM4

Successfully extracted gene data with 54630 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on my examination of the gene identifiers, they are Affymetrix probe IDs 
# and need to be mapped to standard human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 9888255 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for the gene mapping
# From previous output, we can see that the gene expression data uses identifiers like '1007_s_at'
# These correspond to the 'ID' column in the gene annotation dataframe
# 'Gene Symbol' column contains the human gene symbols we need
print("Creating gene mapping from 'ID' to 'Gene Symbol'...")

# 2. Extract the gene mapping dataframe with the two identified columns
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Created gene mapping with {len(gene_mapping_df)} rows")
print("Gene mapping sample:")
print(preview_df(gene_mapping_df))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
print(f"Generated gene expression data with {len(gene_data.index)} unique genes")
print("Gene expression data preview:")
print(preview_df(gene_data))

# 4. Normalize gene symbols
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data.index)} unique genes")

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping from 'ID' to 'Gene Symbol'...


Created gene mapping with 45782 rows
Gene mapping sample:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Converting probe-level measurements to gene expression data...


Generated gene expression data with 21278 unique genes
Gene expression data preview:
{'GSM4830213': [0.221462756, 0.333740771, 0.212150149, 0.959113508, 0.401674867], 'GSM4830214': [0.294765294, 0.325517625, 0.132626891, 0.858857632, 0.533249497], 'GSM4830215': [0.058988549, 0.249811336, 0.19511625999999999, 0.9491526189999999, 0.459077477], 'GSM4830216': [0.267430305, 0.289917976, 0.141253118, 0.9247989059999999, 0.307557851], 'GSM4830217': [0.136383295, 0.203056037, 0.245214999, 1.010768578, 0.401036233], 'GSM4830218': [0.161734939, 0.263107061, 0.394205831, 0.9552827479999999, 0.618241847], 'GSM4830219': [0.056903966, 0.362911552, 0.133408297, 0.9444870059999999, 0.315838575], 'GSM4830220': [0.293876171, 0.291660756, 0.300348483, 0.944939867, 0.393645585], 'GSM4830221': [0.319997162, 0.340124398, 0.424134016, 0.933243335, 0.289862931], 'GSM4830222': [0.209602758, 0.305389136, 0.345813357, 0.9003249409999999, 0.553316236], 'GSM4830223': [0.211825371, 0.264213145, 0.362419479, 0.33431

Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE159472.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# The normalization step was already completed in Step 6, so we can proceed directly to linking

# 2. Link the clinical and genetic data
print("\nLinking clinical and gene expression data...")
try:
    # Load the clinical data that was processed in Step 2
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        is_trait_available = True
    else:
        print(f"Clinical data file not found at {out_clinical_data_file}")
        is_trait_available = False
    
    # Load gene expression data
    if os.path.exists(out_gene_data_file):
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        print(f"Loaded gene expression data with shape: {gene_data.shape}")
        is_gene_available = True
    else:
        print(f"Gene data file not found at {out_gene_data_file}")
        is_gene_available = False
    
    # If both data types are available, proceed with linking
    if is_gene_available and is_trait_available:
        # Link the clinical and gene expression data
        linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
        print(f"Created linked data with shape: {linked_data.shape}")
        
        # Print columns to debug
        print("Columns in linked_data:", linked_data.columns.tolist()[:10], "... and more")
        
        # 3. Handle missing values
        print("\nHandling missing values...")
        # In our dataset, the trait is the first row in the dataframe after linking
        trait_col = linked_data.index[0] if not linked_data.empty else trait
        print(f"Using trait column: {trait_col}")
        
        linked_data = handle_missing_values(linked_data, trait_col=trait_col)
        print(f"After handling missing values, linked data shape: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        print("\nChecking for biased features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_col)
        
        # 5. Conduct final quality validation
        note = f"Dataset contains gene expression data for Large_B-cell_Lymphoma with trait representing DLBCL subtypes (GCB=0, ABC=1)."
        
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=linked_data,
            note=note
        )
        
        # 6. Save linked data if usable
        if is_usable:
            print(f"Dataset is usable. Saving linked data...")
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print(f"Dataset is not usable for {trait} association studies due to bias.")
    else:
        print("Cannot link data: either gene expression or clinical data is missing.")
        # Create a small empty DataFrame as a placeholder for validation
        linked_data = pd.DataFrame()
        is_biased = True
        
        note = "Dataset is missing either gene expression or clinical data for proper linkage."
        
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=linked_data,
            note=note
        )
        
        print(f"Dataset usability for {trait} association studies: {is_usable}")
        
except Exception as e:
    print(f"Error in data linking process: {str(e)}")
    import traceback
    traceback.print_exc()  # Print detailed error information
    # Handle the error case for validation
    linked_data = pd.DataFrame()
    is_biased = True
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available if 'is_gene_available' in locals() else False,
        is_trait_available=is_trait_available if 'is_trait_available' in locals() else False,
        is_biased=is_biased,
        df=linked_data,
        note=f"Error during data processing: {str(e)}"
    )
    print(f"Dataset usability for {trait} association studies: {is_usable}")


Normalizing gene symbols...

Linking clinical and gene expression data...
Loaded clinical data with shape: (1, 180)


Loaded gene expression data with shape: (19845, 180)
Created linked data with shape: (180, 19846)
Columns in linked_data: [0, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT'] ... and more

Handling missing values...
Using trait column: GSM4830213
Error in data linking process: ['GSM4830213']
Abnormality detected in the cohort: GSE159472. Preprocessing failed.
Dataset usability for Large_B-cell_Lymphoma association studies: False


Traceback (most recent call last):
  File "/tmp/ipykernel_63513/691907968.py", line 41, in <module>
    linked_data = handle_missing_values(linked_data, trait_col=trait_col)
  File "/media/techt/DATA/GenoAgent/tools/preprocess.py", line 430, in handle_missing_values
    df = df.dropna(subset=[trait_col])
  File "/home/techt/anaconda3/envs/agent/lib/python3.10/site-packages/pandas/core/frame.py", line 6670, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['GSM4830213']
